In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Kullanılan Kütüphaneler

Aşağıdaki kütüphaneler, model oluşturma, eğitim, değerlendirme ve görselleştirme işlemleri için kullanılır:

1. **os**: Dosya ve dizin işlemleri için kullanılır.
2. **cv2 (OpenCV)**: Görsel işleme için kullanılır. Görsellerin okunması ve boyutlandırılması işlemleri yapılır.
3. **numpy**: Matematiksel işlemler için kullanılan temel kütüphane, veri manipülasyonu için kullanılır.
4. **sklearn.model_selection.train_test_split**: Veriyi eğitim ve test setlerine ayırmak için kullanılır.
5. **tensorflow.keras.preprocessing.image.ImageDataGenerator**: Veri artırma için kullanılır, eğitim verisinin çeşitlendirilmesi amacıyla.
6. **matplotlib.pyplot**: Veri görselleştirme için kullanılır. Eğitim doğruluğu ve kaybı gibi metrikler çizilir.
7. **tensorflow.keras.callbacks.EarlyStopping ve ReduceLROnPlateau**: Modelin eğitimini erken durdurmak ve öğrenme oranını dinamik olarak azaltmak için kullanılır.
8. **tensorflow.keras.utils.to_categorical**: Etiketleri one-hot encoding formatına dönüştürmek için kullanılır.
9. **tensorflow.keras.models.Sequential**: Derin öğrenme modelinin yapılandırılması için kullanılır.
10. **tensorflow.keras.layers.Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization**: Derin öğrenme modelinin katmanlarını tanımlamak için kullanılır.
11. **tensorflow.keras.regularizers.l2**: Modelin aşırı öğrenmesini engellemek için düzenleme teknikleri kullanılır.
12. **sklearn.metrics.confusion_matrix, f1_score, accuracy_score**: Modelin doğruluğunu ve performansını değerlendirmek için kullanılır.
13. **seaborn**: Karışıklık matrisi gibi görselleştirmeler için kullanılır.
14. **keras.models.load_model**: Daha önce eğitilmiş bir modeli yüklemek için kullanılır.



In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.models import load_model





## Görsellerin Yüklenmesi ve Ön İşleme

Bu kod, belirli sınıflara ait görselleri bir klasörden alıp, görselleri yeniden boyutlandırarak normalleştirir ve etiketler ile birlikte bir listeye ekler.

### Adımlar:
1. **Görsel Klasörü ve Sınıflar**: Görsellerin bulunduğu klasör `image_dir` olarak tanımlanır. Çalışılacak sınıflar `classes` listesinde belirtilir.
2. **Görsel ve Etiket Listeleri**: Görsellerin ve etiketlerin tutulacağı boş listeler (`X` ve `y`) oluşturulur.
3. **Görsellerin Yüklenmesi**: 
    - Her bir sınıf için belirtilen alt klasörden ilk 650 görsel alınır.
    - Görseller, `cv2.imread` ile okunur, 224x224 boyutuna yeniden boyutlandırılır ve `255` ile normalleştirilir.
    - Görsel ve etiketler sırasıyla `X` ve `y` listelerine eklenir.
4. **Veri Seti Dönüştürme**: `X` ve `y` listeleri NumPy dizilerine dönüştürülür.
5. **Sonuç**: Veri setinin boyutları ekrana yazdırılır.
6. 
### Çıktı:
- Görsellerin ve etiketlerin boyutları yazdırılır (`X.shape`, `y.shape`).


In [ ]:
# Görsellerin bulunduğu klasör
image_dir = '/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages'

# Kullanılacak sınıflar
classes = ['collie', 'dolphin', 'elephant', 'fox', 'moose', 'rabbit', 'sheep', 'squirrel', 'giant panda', 'polar bear']

# Görsellerin listeleneceği liste
X = []  # Görseller
y = []  # Etiketler

# Her sınıf için sadece ilk 650 görseli alalım
for label_idx, class_name in enumerate(classes):
    class_dir = os.path.join(image_dir, class_name)
    
    # Alt dizinde görsellerin olup olmadığını kontrol et
    if os.path.exists(class_dir):
        images = os.listdir(class_dir)
        
        # İlk 650 görseli al
        selected_images = images[:650]
        
        for img_name in selected_images:
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            
            # Resmi yeniden boyutlandırma
            img_resized = cv2.resize(img, (224, 224))  # 224x224 boyutunda
            img_normalized = img_resized / 255.0  # Normalizasyon
            
            X.append(img_normalized)
            y.append(label_idx)  # Etiketi sayısal hale getirdik
    else:
        print(f"Class {class_name} not found in the directory: {class_dir}")

# NumPy dizisine dönüştürme
X = np.array(X)
y = np.array(y)

# Veri setinin boyutlarını kontrol edelim
print(f"X shape: {X.shape}, y shape: {y.shape}")


## Eğitim ve Test Setlerine Ayırma

Bu kod, görsel veri setini eğitim ve test setlerine ayırır.

### Adımlar:
1. **Veriyi Bölme**: 
    - `train_test_split` fonksiyonu, veri setini eğitim ve test setlerine ayırır.
    - `test_size=0.3` parametresi ile verinin %30'u test setine ayrılır.
    - `random_state=42` parametresi ile rastgelelikin sabitlenmesi sağlanır.
2. **Boyut Kontrolü**: Eğitim ve test setlerinin boyutları `shape` fonksiyonu ile kontrol edilir.

### Çıktı:
- Eğitim ve test setlerinin boyutları ekrana yazdırılır (`X_train.shape`, `X_test.shape`, `y_train.shape`, `y_test.shape`).


In [ ]:
# Veriyi eğitim ve test setlerine ayıralım
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Eğitim ve test setlerinin boyutlarını kontrol edelim
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")



## Etiketleri One-Hot Encoding Formatına Dönüştürme

Bu kod, etiketleri one-hot encoding formatına dönüştürerek her sınıf için ayrı bir sütun oluşturur.

### Adımlar:
1. **One-Hot Encoding**: 
    - `to_categorical` fonksiyonu ile `y_train` ve `y_test` etiketleri, her sınıf için birer sütun oluşturacak şekilde one-hot encoding formatına dönüştürülür.
    - `num_classes=10` parametresi, toplam 10 sınıf olduğunu belirtir.
2. **Boyut Kontrolü**: Yeni etiket dizilerinin boyutları `shape` fonksiyonu ile kontrol edilir.

### Çıktı:
- One-hot encoding işleminden sonra `y_train.shape` ve `y_test.shape` boyutları ekrana yazdırılır.


In [ ]:
# Etiketleri one-hot encoding formatına dönüştürme
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Yeni boyutları kontrol et
print(f"y_train shape after one-hot encoding: {y_train.shape}")
print(f"y_test shape after one-hot encoding: {y_test.shape}")


## Veri Artırma için ImageDataGenerator Kullanımı

Bu kod, eğitim verisini artırmak için `ImageDataGenerator` sınıfını kullanarak çeşitli veri artırma tekniklerini uygular.

### Adımlar:
1. **Veri Artırma Ayarları**: 
    - `ImageDataGenerator` ile görseller üzerinde döndürme, kaydırma, yakınlaştırma, çevirme gibi teknikler uygulanır.
    - Parametreler:
        - `rotation_range=30`: Görselleri 30 dereceye kadar döndürme.
        - `width_shift_range=0.2`: Yatayda %20'lik kaydırma.
        - `height_shift_range=0.2`: Dikeyde %20'lik kaydırma.
        - `shear_range=0.2`: Eğimli kaydırma.
        - `zoom_range=0.2`: Yakınlaştırma.
        - `horizontal_flip=True`: Yatay çevirme.
        - `fill_mode='nearest'`: Yeni boş alanları en yakın pikselle doldurma.
2. **Veri Artırma Uygulama**: 
    - `train_generator` ile eğitim verilerine artırma uygulanır, her seferinde 64 örnek alınır.
3. **Örnek Görsel Gösterimi**:
    - Artırılmış bir görsel `matplotlib` ile görselleştirilir.

### Çıktı:
- Augmente edilmiş bir örnek görsel ekranda gösterilir.


In [ ]:
# Veri artırma için ImageDataGenerator kullanımı
datagen = ImageDataGenerator(
    rotation_range=30,    # Resmi döndürme
    width_shift_range=0.2,  # Yatay kaydırma
    height_shift_range=0.2, # Dikey kaydırma
    shear_range=0.2,      # Eğimli kaydırma
    zoom_range=0.2,       # Yakınlaştırma
    horizontal_flip=True, # Yatay çevirme
    fill_mode='nearest'   # Boş alanı doldurma
)
# Eğitim setine veri artırma uygulama
train_generator = datagen.flow(X_train, y_train, batch_size=64)

# Augmente edilmiş bir örnek görüntü görmek için
import matplotlib.pyplot as plt
augmented_img = next(datagen.flow(X_train, y_train, batch_size=1))[0][0]
plt.imshow(augmented_img)
plt.show()

## Modeli Oluşturma

Bu kod, derin öğrenme modeli oluşturur ve eğitim için derler.

### Adımlar:
1. **Model Yapısı**:
    - `Sequential` model kullanılarak katmanlar sırayla eklenir.
    
2. **Konvolüsyon Katmanları**:
    - 4 adet konvolüsyon katmanı (`Conv2D`) eklenir.
    - Her katmanda `ReLU` aktivasyon fonksiyonu, `L2` regularizasyonu ve `MaxPooling2D` ile havuzlama yapılır.
    - BatchNormalization ile her katmandan sonra normalizasyon uygulanır.

3. **Fully Connected Katmanlar**:
    - `Flatten` katmanı ile 2D çıktılar 1D'ye dönüştürülür.
    - `Dense` katmanları ile tam bağlantılı katmanlar eklenir, her iki katman arasında `Dropout` ile aşırı uyum önlenir.
    - İlk `Dense` katmanında 512 nöron, ikinci `Dense` katmanında ise 256 nöron bulunur.

4. **Çıkış Katmanı**:
    - 10 sınıflı çıkış için `softmax` aktivasyonu kullanılır.

5. **Modelin Derlenmesi**:
    - Adam optimizasyon algoritması ve `categorical_crossentropy` kayıp fonksiyonu kullanılarak model derlenir.
    - Doğruluk metriği izlenir.

6. **Model Özeti**:
    - `model.summary()` fonksiyonu ile modelin yapısı ve katman bilgisi yazdırılır.

### Çıktı:
- Modelin katmanları ve her katmanın parametre sayıları yazdırılır.


In [ ]:
# Modeli oluştur
model = Sequential()

# İlk Konvolüsyon Katmanı
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# İkinci Konvolüsyon Katmanı
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Üçüncü Konvolüsyon Katmanı
model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dördüncü Konvolüsyon Katmanı
model.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fully Connected Katmanlar
model.add(Flatten())  # Katmanı düzleştir
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5)) # Overfitting'i azaltmak için Dropout

model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.7)) # Oranı biraz artırdık

# Çıkış Katmanı (Softmax ile 10 sınıf)
model.add(Dense(10, activation='softmax', kernel_regularizer=l2(0.01)))

# Modeli derle
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00008),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model özeti
model.summary()


## Callbacks Kullanımı ve Model Eğitimi

Bu kod, modelin eğitim sürecini iyileştirmek için `EarlyStopping` ve `ReduceLROnPlateau` callback'lerini kullanır.

### Adımlar:
1. **EarlyStopping**:
    - `monitor='val_loss'`: Doğrulama kaybını takip eder.
    - `patience=5`: 5 epoch boyunca kayıp iyileşmezse eğitimi durdurur.
    - `restore_best_weights=True`: Eğitim durduğunda en iyi ağırlıkları geri yükler.

2. **ReduceLROnPlateau**:
    - `monitor='val_loss'`: Doğrulama kaybına göre öğrenme oranını ayarlar.
    - `factor=0.5`: Öğrenme oranını yarıya indirir.
    - `patience=3`: 3 epoch boyunca iyileşme olmazsa öğrenme oranını azaltır.
    - `min_lr=1e-6`: Öğrenme oranının alt sınırını belirler.

3. **Modeli Eğitme**:
    - `model.fit()` fonksiyonu ile model, eğitim verileri (`X_train`, `y_train`) üzerinde eğitilir.
    - `validation_data=(X_test, y_test)` parametresi ile doğrulama verisi sağlanır.
    - `callbacks=[early_stopping, reduce_lr]` ile callback'ler eklenir.
    - Maksimum 65 epoch, batch boyutu ise 64 olarak belirlenir.

### Çıktı:
- Modelin eğitim süreci, doğrulama verisi ile izlenir ve callback'ler uygulanır.


In [ ]:
# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',  # Doğrulama kaybını takip et
    patience=5,  # İyileşme olmazsa 5 epoch sonra durdur
    restore_best_weights=True  # En iyi ağırlıkları geri yükle
)

# Öğrenme oranını dinamik olarak azalt
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Doğrulama kaybına göre öğrenme oranını güncelle
    factor=0.5,  # Öğrenme oranını yarıya indir
    patience=3,  # 3 epoch boyunca iyileşme olmazsa öğrenme oranını azalt
    min_lr=1e-6  # Minimum öğrenme oranı
)

# Modeli eğitme
history = model.fit(
    X_train,  # Eğitim verileri
    y_train,  # Eğitim etiketleri
    epochs=65,  # Maksimum epoch sayısı
    validation_data=(X_test, y_test),  # Doğrulama verisi
    callbacks=[early_stopping, reduce_lr],  # Callbacks ekleniyor
    batch_size=64,  # Mini-batch boyutu
)


## Eğitim Doğruluğu ve Kaybını Görselleştirme

Bu kod, modelin eğitim sürecindeki doğruluk ve kayıp değerlerini görselleştirir.

### Adımlar:
1. **Eğitim Doğruluğu (Accuracy) Grafiği**:
    - `history.history['accuracy']`: Eğitim doğruluğu.
    - `history.history['val_accuracy']`: Doğrulama doğruluğu.
    - X eksenine epoch sayısı, Y eksenine doğruluk değerleri yerleştirilir.
    - Grafik, eğitim ve doğrulama doğruluğunu aynı anda gösterir.

2. **Eğitim Kaybı (Loss) Grafiği**:
    - `history.history['loss']`: Eğitim kaybı.
    - `history.history['val_loss']`: Doğrulama kaybı.
    - X eksenine epoch sayısı, Y eksenine kayıp değerleri yerleştirilir.
    - Grafik, eğitim ve doğrulama kaybını karşılaştırarak gösterir.

### Çıktı:
- İki grafik, eğitim doğruluğu ve kaybının zaman içindeki değişimini görselleştirir.


In [ ]:
# Eğitim doğruluğunu ve kaybını görselleştirme
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Eğitim kaybını görselleştirme
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()


## Karışıklık Matrisi Görselleştirme

Bu kod, modelin tahminlerinin doğruluğunu analiz etmek için karışıklık matrisini hesaplar ve görselleştirir.

### Adımlar:
1. **One-hot Encoding'den Sayısal Etikete Dönüştürme**:
    - `y_test` birden fazla boyut içeriyorsa (yani, one-hot encoded ise), `np.argmax()` kullanılarak her bir örnek için en yüksek değere sahip olan sınıf etiketi alınır.

2. **Tahminlerin Yapılması**:
    - `model.predict(X_test)` ile test verisi üzerinden tahminler yapılır.
    - `np.argmax()` kullanılarak, tahmin edilen sonuçlar sayısal sınıf etiketlerine dönüştürülür.

3. **Karışıklık Matrisi**:
    - `confusion_matrix(y_test, y_pred)` ile gerçek ve tahmin edilen etiketler arasındaki ilişki hesaplanır.

4. **Karışıklık Matrisi Görselleştirme**:
    - `sns.heatmap()` ile karışıklık matrisi görselleştirilir.
    - `annot=True`: Her hücrede sayısal değerler gösterilir.
    - `fmt='d'`: Değerlerin tam sayı formatında yazdırılması sağlanır.
    - `cmap='Reds'`: Renk paleti belirlenir.
    - Etiketler ve başlıklar eklenir.

### Çıktı:
- Karışıklık matrisi, sınıflar arasındaki tahmin doğruluğunu ve hataları görsel olarak gösterir.


In [ ]:
# Eğer y_test one-hot encoded ise, her bir örnek için en yüksek değeri alarak sayısal etiketlere dönüştür
if len(y_test.shape) > 1:
    y_test = np.argmax(y_test, axis=1)

# Gerçek ve tahmin edilen etiketler
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Tahminleri sınıf etiketlerine dönüştür

# Karışıklık Matrisi
cm = confusion_matrix(y_test, y_pred)

# Etiket isimleri
classes = ['collie', 'dolphin', 'elephant', 'fox', 'moose', 'rabbit', 'sheep', 'squirrel', 'giant panda', 'polar bear']

# Grafikleştirme
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', xticklabels=classes, yticklabels=classes)

# Başlık ve etiketler
plt.title('Confusion Matrix')
plt.ylabel('Gerçek Etiketler')
plt.xlabel('Tahmin Edilen Etiketler')

plt.show()


## Modeli Değerlendirme: F1 Skoru Hesaplama

Bu kod, modelin performansını değerlendirmek için F1 skorunu hesaplar.

### Adımlar:
1. **Tahminlerin Yapılması**:
    - `model.predict(X_test)` ile test verisi üzerinden tahminler yapılır.
    - `np.argmax()` ile tahmin edilen sınıf etiketleri, çok sınıflı sınıflandırma için alınır.

2. **F1 Skoru Hesaplama**:
    - `f1_score(y_test, y_pred_classes, average='weighted')`: Gerçek etiketler (`y_test`) ve tahmin edilen etiketler (`y_pred_classes`) arasındaki F1 skoru hesaplanır.
    - `average='weighted'`: F1 skoru tüm sınıfların ağırlıklı ortalamasını alır.

### Çıktı:
- Hesaplanan F1 skoru, modelin genel doğruluğunu ve dengesini değerlendirmek için kullanılır.


In [ ]:
# Modeli değerlendirme
y_pred = model.predict(X_test)  # Tahminler
y_pred_classes = np.argmax(y_pred, axis=1)  # Çok sınıflı sınıflandırma için sınıf etiketlerini al

# F1 Skoru hesaplama
f1 = f1_score(y_test, y_pred_classes, average='weighted')  # 'weighted' tüm sınıfların ağırlıklı ortalamasını alır

print(f"F1 Score: {f1}")


## Modeli Kaydetme

Bu kod, eğitilmiş modelin `.h5` formatında kaydedilmesini sağlar.

### Adımlar:
1. **Modeli Kaydetme**:
    - `model.save('trained_model.h5')`: Modelin ağırlıkları, yapılandırması ve eğitim durumu `.h5` uzantılı bir dosyaya kaydedilir.
  
### Çıktı:
- `trained_model.h5` adıyla kaydedilen dosya, modelin ileride tekrar kullanılabilmesi için saklanır.


In [ ]:
model.save('trained_model.h5')


# Görsel Manipülasyonu ve Veri Hazırlığı

Bu kod parçası, belirli bir dizindeki görselleri yükler, her birini belirli bir boyutta yeniden boyutlandırır, normalizasyon uygular ve ardından ışık manipülasyonu (parlaklık artışı) uygular.

### 1. `get_manipulated_images` Fonksiyonu:
- **Kullanım**: Görüntüye parlaklık ekler.
- **Girdi**: Görüntü (image), parlaklık değeri (brightness).
- **Çıktı**: Parlaklık artırılmış görüntü.

### 2. Görselleri Yükleme ve Manipüle Etme:
- **İşlem**:
  - `classes` dizisindeki her sınıf için görseller yüklenir.
  - Görseller, 224x224 boyutlarına yeniden boyutlandırılır.
  - Görseller normalize edilir (0-255 arası değerler 0-1 arası dönüştürülür).
  - Görüntüye parlaklık manipülasyonu uygulanır.
  
- **Çıktı**: Manipüle edilmiş görseller `X_manipulated` ve etiketler `y_manipulated` listelerinde toplanır.

### 3. Veri Seti:
- **X_manipulated**: Manipüle edilmiş görsellerin dizisi.
- **y_manipulated**: Etiketler dizisi.
- **Boyut**: `X_manipulated.shape` ve `y_manipulated.shape` ile veri setinin boyutu yazdırılır.



In [ ]:
def get_manipulated_images(image, brightness=50):
    # Görüntüye ışık ekleme
    manipulated_img = cv2.convertScaleAbs(image, alpha=1, beta=brightness)
    return manipulated_img

# Görselleri yükleme ve manipüle etme
X_manipulated = []
y_manipulated = []

for label_idx, class_name in enumerate(classes):
    class_dir = os.path.join(image_dir, class_name)
    
    if os.path.exists(class_dir):
        images = os.listdir(class_dir)
        selected_images = images[:650]
        
        for img_name in selected_images:
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            img_resized = cv2.resize(img, (224, 224))  # 224x224 boyutunda
            img_normalized = img_resized / 255.0  # Normalizasyon
            
            # Işık manipülasyonu
            manipulated_img = get_manipulated_images(img_normalized)
            X_manipulated.append(manipulated_img)
            y_manipulated.append(label_idx)

X_manipulated = np.array(X_manipulated)
y_manipulated = np.array(y_manipulated)

print(f"Manipüle edilmiş görsel setinin boyutu: X: {X_manipulated.shape}, y: {y_manipulated.shape}")


# Renk Sabitliği ve Görsel Manipülasyonu

Bu kod parçası, belirli bir dizindeki görselleri yükler, her birini belirli bir boyutta yeniden boyutlandırır, normalizasyon uygular ve ardından renk sabitliği (Gray World algoritması) uygular.

### 1. `get_wb_images` Fonksiyonu:
- **Kullanım**: Görüntüye renk sabitliği (Gray World) algoritması uygular.
- **İşlem**:
  - Görüntüdeki her renk kanalının (kırmızı, yeşil, mavi) ortalama değeri hesaplanır.
  - Bu değerler, her renk kanalını ortalama değere getirecek şekilde ölçeklendirilir.
- **Çıktı**: Renk sabitliği uygulanmış görüntü.

### 2. Görselleri Yükleme ve Manipüle Etme:
- **İşlem**:
  - `classes` dizisindeki her sınıf için görseller yüklenir.
  - Görseller, 224x224 boyutlarına yeniden boyutlandırılır.
  - Görseller normalize edilir (0-255 arası değerler 0-1 arası dönüştürülür).
  - Renk sabitliği uygulanır.
  
- **Çıktı**: Renk sabitliği uygulanmış görseller `X_wb` ve etiketler `y_wb` listelerinde toplanır.

### 3. Veri Seti:
- **X_wb**: Renk sabitliği uygulanmış görsellerin dizisi.
- **y_wb**: Etiketler dizisi.
- **Boyut**: `X_wb.shape` ve `y_wb.shape` ile veri setinin boyutu yazdırılır.



In [ ]:
def get_wb_images(image):
    # Renk sabitliği algoritması (Gray World)
    avg_b = np.mean(image[:, :, 0])
    avg_g = np.mean(image[:, :, 1])
    avg_r = np.mean(image[:, :, 2])
    
    avg = (avg_b + avg_g + avg_r) / 3
    image[:, :, 0] = image[:, :, 0] * (avg / avg_b)
    image[:, :, 1] = image[:, :, 1] * (avg / avg_g)
    image[:, :, 2] = image[:, :, 2] * (avg / avg_r)
    
    return image

# Manipüle edilmiş görsellere renk sabitliği ekleyelim
X_wb = []
y_wb = []

for label_idx, class_name in enumerate(classes):
    class_dir = os.path.join(image_dir, class_name)
    
    if os.path.exists(class_dir):
        images = os.listdir(class_dir)
        selected_images = images[:650]
        
        for img_name in selected_images:
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            img_resized = cv2.resize(img, (224, 224))  # 224x224 boyutunda
            img_normalized = img_resized / 255.0  # Normalizasyon
            
            # Renk sabitliği uygula
            wb_img = get_wb_images(img_normalized)
            X_wb.append(wb_img)
            y_wb.append(label_idx)

X_wb = np.array(X_wb)
y_wb = np.array(y_wb)

print(f"Renk sabitliği uygulanmış görsel setinin boyutu: X: {X_wb.shape}, y: {y_wb.shape}")


# Modeli Test Etme ve Doğrulama

Bu kod, daha önce eğitilmiş bir modeli yükler ve manipüle edilmiş ve renk sabitliği uygulanmış görsel setleri üzerinde test eder. Test sonuçları doğruluk oranı olarak raporlanır.

### 1. Model Yükleme:
- **Yöntem**: `load_model('trained_model.h5')`
  - `trained_model.h5` dosyasındaki eğitilmiş model yüklenir.

### 2. Test Fonksiyonu:
- **Fonksiyon**: `test_model(model, X_test, y_test)`
  - **Girdi**:
    - `model`: Eğitilmiş model.
    - `X_test`: Test görselleri.
    - `y_test`: Gerçek etiketler.
  - **İşlem**:
    - Model, test verisi (`X_test`) üzerinde tahmin yapar.
    - Tahminler, sınıf etiketlerine dönüştürülür.
    - Gerçek etiketlerle karşılaştırılarak doğruluk hesaplanır.
  - **Çıktı**: Test doğruluğu yazdırılır.

### 3. Test İşlemleri:
- **Manipüle Edilmiş Görseller ile Test**: `X_manipulated` ve `y_manipulated` veri setleri ile model test edilir.
- **Renk Sabitliği Uygulanmış Görseller ile Test**: `X_wb` ve `y_wb` veri setleri ile model test edilir.

### 4. Sonuç:
- Her iki test seti için doğruluk oranları ekrana yazdırılır.


In [ ]:
# Eğitilmiş modelinizi yükleyin
model = load_model('trained_model.h5')

# Modeli test etme
def test_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Test doğruluğu: {accuracy * 100:.2f}%")

# Manipüle edilmiş set ile testi yapalım
test_model(model, X_manipulated, y_manipulated)

# Renk sabitliği uygulanmış set ile testi yapalım
test_model(model, X_wb, y_wb)

